In [ ]:
from openai import OpenAI
from config import OPENAI_API_KEY
import json

client = OpenAI(api_key=OPENAI_API_KEY)

# ---- Function that will handle the event when AI calls it ----
def create_calendar_event(title, date, time, attendee):
    print("\n[ACTION] Creating Calendar Event...")
    print(f"Title: {title}")
    print(f"Date : {date}")
    print(f"Time : {time}")
    print(f"Attendee: {attendee}")
    print("-" * 40)


# ---- User conversation input ----
messages = [
    {"role": "system", "content": "You are a helpful assistant that can create calendar events."},
    {"role": "user", "content": "Schedule a meeting tomorrow at 3 PM with John about project updates."}
]


# ---- Tool definition (JSON schema that AI must fill) ----
tools = [
    {
        "type": "function",
        "name": "create_event",
        "description": "Create a calendar event with title, date, time, and attendee.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "date": {"type": "string"},
                "time": {"type": "string"},
                "attendee": {"type": "string"}
            },
            "required": ["title", "date", "time", "attendee"]
        }
    }
]

# ---- Stream the response ----
stream = client.responses.create(
    model="gpt-4.1-mini",
    input=messages,
    tools=tools,
    tool_choice={"type": "function", "name": "create_event"},
    stream=True
)

# ---- Handle streamed events ----
full_function_call = ""

# Collect text while streaming
for event in stream:
    if event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)
    elif event.type == "response.completed":
        print("\n\n[INFO] Streaming complete.\n")

In [ ]:
print(event.response.output[0])

ResponseFunctionToolCall(arguments='{"title":"Meeting about project updates","date":"2024-04-28","time":"15:00","attendee":"John"}', call_id='call_eo732tuouyVtV7ZfeiILV8m2', name='create_event', type='function_call', id='fc_0b3810b734f7a79600690b1ed4069881949553ee34f82aa7f3', status='completed')


In [21]:
print(event.response.output[0].type)

function_call


In [ ]:
print(event.response.temperature)
print(event.response.tool_choice.name)
print(event.response.tool_choice.type)

print(event.response.tools[0].name)
print(event.response.tools[0].parameters)
print(event.response.tools[0].strict)
print(event.response.tools[0].description)

1.0
create_event
function
create_event
{'type': 'object', 'properties': {'title': {'type': 'string'}, 'date': {'type': 'string'}, 'time': {'type': 'string'}, 'attendee': {'type': 'string'}}, 'required': ['title', 'date', 'time', 'attendee'], 'additionalProperties': False}
True
Create a calendar event with title, date, time, and attendee.


In [ ]:
tool_call = event.response.output[0]  # first output object (the function call)

# Parse the arguments (they are stored as a JSON string)
args = json.loads(tool_call.arguments)

# Call your function with the parsed arguments
create_calendar_event(
    title=args["title"],
    date=args["date"],
    time=args["time"],
    attendee=args["attendee"]
)

# Example Output

# [ACTION] Creating Calendar Event...
# Title: Meeting about project updates
# Date : 2024-04-28
# Time : 15:00
# Attendee: John
# ----------------------------------------



[ACTION] Creating Calendar Event...
Title: Meeting about project updates
Date : 2024-04-28
Time : 15:00
Attendee: John
----------------------------------------


In [15]:
print(tool_call.name)
print(tool_call.type)
print(tool_call.status)

create_event
function_call
completed
